In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from keras.preprocessing.text import  one_hot , Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation , Dropout , Dense
from keras.models import Sequential
from keras.layers import Flatten , GlobalMaxPooling1D , Embedding ,Conv1D ,LSTM
from sklearn.model_selection import train_test_split

C:\Users\hp\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [30]:
df = pd.read_csv("a1_IMDB_Dataset.csv")

In [31]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [71]:
df["sentiment"] = df["sentiment"].map({"positive":1 ,"negative":0})

In [72]:
df["sentiment"]

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64

In [36]:
df['review']

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

In [32]:
df.shape

(50000, 2)

In [33]:
import re

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
  return TAG_RE.sub('',text)

def remove_urls(text):
    clean_text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)  # Remove URLs
    return clean_text

In [37]:
df['review'] = df['review'].apply(lambda x : remove_tags(x))
df['review'] = df['review'].apply(lambda x : remove_urls(x))

In [38]:
def preprocess_text(sen):
  sentence = sen.lower()
  sentence = remove_tags(sentence)
  sentence = re.sub('[^a-zA-Z]', ' ' , sentence)
  sentence = re.sub(r"\s+[a-zA-Z]\s+" ,' ' ,sentence)
  sentence = re.sub(r'\s+' , ' ',sentence)
  pattern = re.compile(r'\b('+r'|'.join(stopwords.words('english'))+r')\b\s*')
  sentence = pattern.sub('',sentence)
  return sentence

In [39]:
df["review"] = df["review"].apply(lambda x : preprocess_text(x))

In [40]:
df["review"]

0        one reviewers mentioned watching oz episode ho...
1        wonderful little production filming technique ...
2        thought wonderful way spend time hot summer we...
3        basically family little boy jake thinks zombie...
4        petter mattei love time money visually stunnin...
                               ...                        
49995    thought movie right good job creative original...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    catholic taught parochial elementary schools n...
49998    going disagree previous comment side maltin on...
49999    one expects star trek movies high art fans exp...
Name: review, Length: 50000, dtype: object

In [73]:
X = df.iloc[: , :-1]
y = df.iloc[ :, -1]

In [75]:
y

0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64

In [74]:
X

,review
0,one reviewers mentioned watching oz episode ho...
1,wonderful little production filming technique ...
2,thought wonderful way spend time hot summer we...
3,basically family little boy jake thinks zombie...
4,petter mattei love time money visually stunnin...
...,...
49995,thought movie right good job creative original...
49996,bad plot bad dialogue bad acting idiotic direc...
49997,catholic taught parochial elementary schools n...
49998,going disagree previous comment side maltin on...


In [76]:
X_train ,X_test  ,y_train , y_test = train_test_split(X ,y ,test_size=0.2  , random_state=42)

In [77]:
X_train.shape

(40000, 1)

In [78]:
X_train

,review
39087,kept asking many fights screaming matches swea...
30893,watch entire movie could watch entire movie st...
45278,touching love story reminiscent mood love draw...
16398,latter day fulci schlocker totally abysmal con...
13653,first firmly believe norwegian movies continua...
...,...
11284,shadow magic recaptures joy amazement first m...
44732,found movie quite enjoyable fairly entertainin...
38158,avoid one terrible movie exciting pointless mu...
860,production quite surprise absolutely love obsc...


In [79]:
X_test.shape

(10000, 1)

In [80]:
y_train.shape

(40000,)

In [81]:
y_test.shape

(10000,)

In [82]:
df_train = pd.concat([X_train , y_train] , axis = 1)

In [83]:
df_test = pd.concat([X_test , y_test] , axis=1)

In [84]:
df_train.shape

(40000, 2)

In [85]:
df_test.shape

(10000, 2)

In [86]:
tokenizer1 = Tokenizer(num_words = 30)
tokenizer1.fit_on_texts(X_train)
tokenizer1.fit_on_texts(X_test)

In [87]:
x_train = tokenizer1.texts_to_sequences(X_train)
x_test = tokenizer1.texts_to_sequences(X_test)

In [88]:
x_test

[[1]]

In [89]:
from keras.layers import Input, Embedding, LSTM, Dropout, Dense
from keras.models import Model

inp1 = Input(shape=(100,)) 

max_features = 10000 
emb_size = 128
emb1 = Embedding(max_features, emb_size)(inp1)


layer1 = LSTM(60)(emb1)


layer2 = Dropout(0.1)(layer1)


out1 = Dense(1, activation='sigmoid')(layer2)


model = Model(inputs=inp1, outputs=out1)


In [90]:
model.compile(loss='binary_crossentropy' ,optimizer='adam',metrics=['accuracy'])

In [91]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding_3 (Embedding)     (None, 100, 128)          1280000   
                                                                 
 lstm_3 (LSTM)               (None, 60)                45360     
                                                                 
 dropout_3 (Dropout)         (None, 60)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 61        
                                                                 
Total params: 1325421 (5.06 MB)
Trainable params: 1325421 (5.06 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [93]:
model.fit(x_train ,y_train , epochs = 5 ,validation_data=(x_test, y_test))

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'int\'>"})'}), <class 'pandas.core.series.Series'>